# Train a $\pi^{+}$ vs. $e^{+}$ model using a deep neural network and a BDT

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

This notebook was tested with Keras v2.0.6

In [24]:
from functools import partial
import h5py
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.layers import (Dense, Reshape, Conv2D, LeakyReLU, BatchNormalization,
                          LocallyConnected2D, Activation, ZeroPadding2D,
                          Dropout, Lambda, Flatten, Input, AlphaDropout, add)

from keras.layers.merge import concatenate, multiply
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import matplotlib
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, classification_report

In [3]:
from calodata.features import extract_features

In [4]:
def load_calodata(fpath):
    with h5py.File(fpath, 'r') as h5:
        data = [h5['layer_{}'.format(i)][:] for i in xrange(3)]
    return data

In [5]:
def build_model(image, selu=True, bn=True):
    x = Conv2D(64, (2, 2), padding='same')(image)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)

    x = ZeroPadding2D((1, 1))(image)
    x = LocallyConnected2D(8 * 4, (3, 3), padding='valid', strides=(1, 2))(x)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)

    x = ZeroPadding2D((1, 1))(x)
    x = LocallyConnected2D(16 * 4, (2, 2), padding='valid')(x)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)

    x = ZeroPadding2D((1, 1))(x)
    x = LocallyConnected2D(32 * 4, (2, 2), padding='valid', strides=(1, 2))(x)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)

    x = Flatten()(x)

    return x

In [6]:
# CLASS_ONE = 'gamma'
# CLASS_TWO = 'eplus'

CLASS_ONE = 'piplus'
CLASS_TWO = 'eplus'

In [7]:
concat = partial(np.concatenate, axis=0)

In [8]:
c1 = load_calodata('../data/{}.hdf5'.format(CLASS_ONE))
c2 = load_calodata('../data/{}.hdf5'.format(CLASS_TWO))
data = map(concat, zip(c1, c2))

labels = np.array([1] * c1[0].shape[0] + [0] * c2[0].shape[0])

In [9]:
features = extract_features(data) # shower shapes

In [10]:
features.shape

(200000, 20)

In [11]:
# random shuffle
np.random.seed(0)
ix = np.array(range(len(labels)))
np.random.shuffle(ix)

# number of examples to train on
nb_train = int(0.7 * len(ix))

# train test split
ix_train = ix[:nb_train]
ix_test = ix[nb_train:]

features_train = features[ix_train]
data_train = [np.expand_dims(d[ix_train], -1) / 1000. for d in data]
labels_train = labels[ix_train]

features_test = features[ix_test]
data_test = [np.expand_dims(d[ix_test], -1) / 1000. for d in data]
labels_test = labels[ix_test]

In [12]:
raveled_train = np.concatenate([d.reshape(d.shape[0], -1) for d in data_train], axis=-1)
raveled_test = np.concatenate([d.reshape(d.shape[0], -1) for d in data_test], axis=-1)

# LAGAN-style discriminator

In [12]:
shapes = [d.shape[1:] for d in data_train]

x = [Input(shape=sh) for sh in shapes]

# h = concatenate(map(partial(build_res_model), x))
h = concatenate(map(partial(build_model), x))

h = Dense(256)(h)
h = Activation('relu')(h)
h = Dropout(0.5)(h)

y = Dense(1, activation='sigmoid')(h)

image_dnn = Model(x, y)

image_dnn.compile('adam', 'binary_crossentropy', metrics=['acc'])

In [13]:
image_dnn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 96, 1)      0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 12, 12, 1)     0                                            
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, 12, 6, 1)      0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 5, 98, 1)      0           input_1[0][0]                    
___________________________________________________________________________________________

In [25]:
callbacks = [
    EarlyStopping(verbose=True, patience=7, monitor='val_loss'),
    ModelCheckpoint('{}vs{}-chkpt.h5'.format(CLASS_ONE, CLASS_TWO),
                    monitor='val_loss', verbose=True, save_best_only=True),
]

In [ ]:
try:
    image_dnn.fit(data_train, labels_train, callbacks=callbacks, verbose=True,
                  validation_split=0.3, batch_size=128, epochs=100)
except KeyboardInterrupt:
    print 'ending early'

Train on 98000 samples, validate on 42000 samples
Epoch 1/100
74880/98000 [=====================>........] - ETA: 17s - loss: 0.0603 - acc: 0.9850

In [ ]:
image_dnn.load_weights('{}vs{}-chkpt.h5'.format(CLASS_ONE, CLASS_TWO))
image_dnn.save_weights('{}vs{}-final.h5'.format(CLASS_ONE, CLASS_TWO))

In [ ]:
image_dnn.load_weights('{}vs{}-final.h5'.format(CLASS_ONE, CLASS_TWO))

In [ ]:
with h5py.File('{}vs{}-split-indices.h5'.format(CLASS_ONE, CLASS_TWO), 'w') as h5:
    h5['train'] = ix_train
    h5['test'] = ix_test

In [ ]:
yhat_image_dnn = image_dnn.predict(data_test, verbose=True).ravel()

# Train a DNN on shower shapes

In [ ]:
def build_feature_dnn(x):

    h = Dense(256)(x)
    h = Dropout(0.2)(LeakyReLU()(h))
    h = BatchNormalization()(h)

    h = Dense(256)(h)
    h = Dropout(0.5)(LeakyReLU()(h))
    h = BatchNormalization()(h)

    h = Dense(256)(h)
    h = Dropout(0.5)(LeakyReLU()(h))
    h = BatchNormalization()(h)
    
    h = Dense(256)(h)
    h = Dropout(0.5)(LeakyReLU()(h))
    h = BatchNormalization()(h)

    h = Dense(32)(h)
    h = Dropout(0.5)(LeakyReLU()(h))

    h = Dense(1)(h)
    y = Activation('sigmoid')(h)
    

    return y

In [ ]:
x = Input(shape=(features_train.shape[1], ))
feature_dnn = Model(x, build_feature_dnn(x))
feature_dnn.compile('adam', 'binary_crossentropy', metrics=['acc'])

In [ ]:
callbacks = [
    EarlyStopping(verbose=True, patience=7, monitor='val_loss'),
    ModelCheckpoint('{}vs{}-features-chkpt.h5'.format(CLASS_ONE, CLASS_TWO),
                    monitor='val_loss', verbose=True, save_best_only=True),
]

In [ ]:
try:
    feature_dnn.fit(features_train / features_train.max(axis=0)[np.newaxis, :], labels_train, callbacks=callbacks, verbose=True,
                  validation_split=0.3, batch_size=128, epochs=100)
except KeyboardInterrupt:
    print 'ending early'

In [ ]:
feature_dnn.load_weights('{}vs{}-features-chkpt.h5'.format(CLASS_ONE, CLASS_TWO))
feature_dnn.save_weights('{}vs{}-features-final.h5'.format(CLASS_ONE, CLASS_TWO))

In [ ]:
feature_dnn.load_weights('{}vs{}-features-final.h5'.format(CLASS_ONE, CLASS_TWO))

In [ ]:
yhat_feature_dnn = feature_dnn.predict(features_test / features_test.max(axis=0)[np.newaxis, :], verbose=True).ravel()

# Train a BDT on shower shapes

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
base_clf = GradientBoostingClassifier(verbose=2)
parameters = {
    'n_estimators':[100, 200, 300],
    'max_depth':[3, 5]
}

In [ ]:
clf.best_params_

In [ ]:
clf = GridSearchCV(base_clf, parameters, n_jobs=7)

In [ ]:
clf.fit(features_train, labels_train)

In [ ]:
yhat_feature_bdt = clf.predict_proba(features_test)[:, 1].ravel()


# Train simple DNN on pix

In [ ]:
def build_simple_dnn(x):

    h = Dense(512)(x)
    h = Dropout(0.2)(LeakyReLU()(h))
    h = BatchNormalization()(h)

    h = Dense(512)(h)
    h = Dropout(0.5)(LeakyReLU()(h))
    h = BatchNormalization()(h)

    h = Dense(256)(h)
    h = Dropout(0.5)(LeakyReLU()(h))
    h = BatchNormalization()(h)
    
    h = Dense(256)(h)
    h = Dropout(0.5)(LeakyReLU()(h))
    h = BatchNormalization()(h)

    h = Dense(32)(h)
    h = Dropout(0.5)(LeakyReLU()(h))

    h = Dense(1)(h)
    y = Activation('sigmoid')(h)
    

    return y

In [ ]:
x = Input(shape=(raveled_train.shape[1], ))
raveled_dnn = Model(x, build_simple_dnn(x))
raveled_dnn.compile('adam', 'binary_crossentropy', metrics=['acc'])

In [ ]:
callbacks = [
    EarlyStopping(verbose=True, patience=7, monitor='val_loss'),
    ModelCheckpoint('{}vs{}-raveled-chkpt.h5'.format(CLASS_ONE, CLASS_TWO),
                    monitor='val_loss', verbose=True, save_best_only=True),
]

In [ ]:
try:
    raveled_dnn.fit(raveled_train, labels_train, callbacks=callbacks, verbose=True,
                          validation_split=0.3, batch_size=128, epochs=100)
except KeyboardInterrupt:
    print 'ending early'

In [ ]:
raveled_dnn.load_weights('{}vs{}-raveled-chkpt.h5'.format(CLASS_ONE, CLASS_TWO))
raveled_dnn.save_weights('{}vs{}-raveled-final.h5'.format(CLASS_ONE, CLASS_TWO))

In [ ]:
raveled_dnn.load_weights('{}vs{}-raveled-final.h5'.format(CLASS_ONE, CLASS_TWO))

In [ ]:
yhat_raveled_dnn = raveled_dnn.predict(raveled_test, verbose=True).ravel()

# Train DenseNet

In [14]:
% cd keras-contrib/keras_contrib/applications/
from densenet import DenseNet as build_densenet
% cd ../../../

/Users/mp744/Documents/CERN/external_dl_work/caloGAN/classification/keras-contrib/keras_contrib/applications
/Users/mp744/Documents/CERN/external_dl_work/caloGAN/classification


In [ ]:
# to support 1 channel images, just comment out the input_shape = _obtain_input_shape part
# just set the input_shape when you build the model (risky without checks)

In [17]:
dnet = build_densenet(weights=None, classes=1, activation='sigmoid', input_shape=(12, 12, 1))

densenet.py:507: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg..., activation="sigmoid", bias_regularizer=<keras.reg...)`


In [28]:
#dnet.summary()

In [18]:
#from keras.utils.vis_utils import plot_model

In [42]:
#plot_model(dnet, to_file='dnet.png', show_shapes=True)

In [44]:
#from IPython.display import Image
#Image(filename='dnet.png') 

In [23]:
dnet.compile('adam', 'binary_crossentropy')

In [29]:
try:
    dnet.fit(data_train[1], labels_train, callbacks=callbacks, verbose=True,
                  validation_split=0.3, batch_size=128, epochs=1)
except KeyboardInterrupt:
    print 'ending early'

Train on 98000 samples, validate on 42000 samples
Epoch 1/1
 9088/98000 [=>............................] - ETA: 3308s - loss: 1.3035 

In [33]:
dnet = build_densenet(weights=None, classes=1, activation='sigmoid', input_shape=(12, 6, 1)) # ugly shapes, but ok

In [41]:
dnet = build_densenet(weights=None, classes=1, activation='sigmoid', input_shape=(3, 96, 1), nb_dense_block=2)
# it doesn't work with nb_dense_block=3

In [ ]:
yhat_image_dnn.max()

In [ ]:
plt.figure(figsize=(10, 10))
bins = np.linspace(0, 1, 100)
plt.hist(yhat_image_dnn[labels_test == 1], histtype='step', bins=bins, label=r'$\pi^{+}$, Image DNN')
plt.hist(yhat_image_dnn[labels_test == 0], histtype='step', bins=bins, label=r'$e^{+}$, Image DNN')
plt.hist(yhat_feature_dnn[labels_test == 1], histtype='step', bins=bins, label=r'$\pi^{+}$, Feature DNN')
plt.hist(yhat_feature_dnn[labels_test == 0], histtype='step', bins=bins, label=r'$e^{+}$, Feature DNN')
plt.legend(loc='upper right')
plt.yscale('log')
# plt.xlim((0.99999, 1))
# plt.xscale('log')
plt.xlabel(r'$\mathbb{P}[\pi^{+}]$')

In [ ]:
fpr_image_dnn, tpr_image_dnn, _ = roc_curve(labels_test, yhat_image_dnn)
fpr_raveled_dnn, tpr_raveled_dnn, _ = roc_curve(labels_test, yhat_raveled_dnn)
fpr_feature_dnn, tpr_feature_dnn, _ = roc_curve(labels_test, yhat_feature_dnn)
fpr_feature_bdt, tpr_feature_bdt, _ = roc_curve(labels_test, yhat_feature_bdt)

In [ ]:
with h5py.File('{}-vs-{}-outputs.h5'.format(CLASS_ONE, CLASS_TWO), 'w') as h5:
    h5['y'] = labels_test
    h5['nn_image'] = yhat_image_dnn
    h5['nn_raveled'] = yhat_raveled_dnn
    h5['nn_showershapes'] = yhat_feature_dnn
    h5['bdt_showershapes'] = yhat_feature_bdt

In [ ]:
slices = np.linspace(0, 1, 10000000)

In [ ]:
tpr, _ = np.histogram(yhat[labels_test == 1], bins=slices)
fpr, _ = np.histogram(yhat[labels_test == 0], bins=slices)

In [ ]:
tpr = np.cumsum(tpr[::-1])[::-1]
fpr = np.cumsum(fpr[::-1])[::-1]

In [ ]:
(tpr / float(tpr[0])).min()

In [ ]:
tpr

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(tpr_image_dnn, 1 / fpr_image_dnn, label='DNN on Calorimeter Hits')
plt.plot(tpr_feature_dnn, 1 / fpr_feature_dnn, label='DNN on Shower Shapes')
plt.yscale('log')
plt.grid('on', 'both')
# plt.xlim((0.9, 1))
plt.xlabel('{} Efficiency'.format(CLASS_ONE))
plt.ylabel('{} Background Rejection'.format(CLASS_TWO))
plt.legend(fontsize=20, loc='lower left')

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(tpr_image_dnn, fpr_image_dnn, label='Deep NN on Calorimeter Hits')
plt.plot(tpr_raveled_dnn, fpr_raveled_dnn, label='FCNN on Calorimeter Hits')
plt.plot(tpr_feature_dnn, fpr_feature_dnn, label='DNN on Shower Shapes')
plt.plot(tpr_feature_bdt, fpr_feature_bdt, label='BDT on Shower Shapes')
# plt.plot(tpr_raveled_dnn, fpr_raveled_dnn, label='Simple DNN on Calorimeter Hits')
plt.yscale('log')
plt.grid('on', 'both')
plt.xlim((0.95, 1))
plt.xlabel('{} Efficiency'.format(CLASS_ONE))
plt.ylabel('{} Background Efficienct'.format(CLASS_TWO))
plt.legend(fontsize=20, loc='upper left')

In [ ]:
from scipy import interpolate
from matplotlib import gridspec


In [ ]:
def uix(a):
    return np.unique(a, return_index=True)[1]

In [ ]:
eff_interp = np.linspace(0.05, 1, 100)
# interp_image_dnn = np.interp(eff_interp, tpr_image_dnn, fpr_image_dnn)
# interp_feature_dnn = np.interp(eff_interp, tpr_feature_dnn, fpr_feature_dnn)
# interp_raveled_dnn = np.interp(eff_interp, tpr_raveled_dnn, fpr_raveled_dnn)

interp_image_dnn = interpolate.interp1d(tpr_image_dnn[uix(tpr_image_dnn)], fpr_image_dnn[uix(tpr_image_dnn)], kind='cubic')(eff_interp)
interp_feature_dnn = interpolate.interp1d(tpr_feature_dnn[uix(tpr_feature_dnn)], fpr_feature_dnn[uix(tpr_feature_dnn)], kind='cubic')(eff_interp)
# interp_raveled_dnn = interpolate.interp1d(tpr_raveled_dnn[uix(tpr_raveled_dnn)], fpr_raveled_dnn[uix(tpr_raveled_dnn)], kind='cubic')(eff_interp)

kern_size = 20
kern = [1 / float(kern_size)] * kern_size
interp_image_dnn = np.convolve(interp_image_dnn, kern, mode='same')
interp_feature_dnn = np.convolve(interp_feature_dnn, kern, mode='same')
# interp_raveled_dnn = np.convolve(interp_raveled_dnn, kern, mode='same')

In [ ]:
plt.figure(figsize=(7, 10))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 2]) 
ax = plt.subplot(gs[0])
plt.plot(tpr_image_dnn, fpr_image_dnn, label='ResNet on Calorimeter Hits')
plt.plot(tpr_feature_dnn, fpr_feature_dnn, label='DNN on Shower Shapes')
# plt.plot(tpr_raveled_dnn, fpr_raveled_dnn, label='Simple DNN on Calorimeter Hits')
plt.yscale('log')
plt.grid('on', 'both')
plt.xlim((0, 1))
plt.xlabel(r'$\pi^{+}$ efficiency')
plt.ylabel(r'$e^{+}$ efficiency')
plt.legend(loc='upper left')

plt.subplot(gs[1], sharex=ax)
plt.plot(eff_interp, interp_feature_dnn / interp_image_dnn, label='ResNet on Calorimeter Hits')
plt.plot(eff_interp, interp_feature_dnn / interp_feature_dnn, label='DNN on Shower Shapes')
# plt.plot(eff_interp, interp_feature_dnn / interp_raveled_dnn, label='Simple DNN on Calorimeter Hits')
# plt.yscale('log')
plt.grid('on', 'both')
# plt.xlim((0.99, 1))
# plt.xlabel(r'$\pi^{+}$ efficiency')
plt.ylabel(r'Ratio of $e^{+}$ efficiencies of DNN on Shower Shapes to X')
plt.legend(loc='upper left')